In [332]:
import pandas as pd
import numpy as np

In [333]:
cctv = pd.read_csv('서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv',
                    skiprows=1, encoding='euc-kr')
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289


In [334]:
df = pd.read_csv('서울시주민등록인구(구별)통계.txt', sep='\t', skiprows=1)
df.head(3)

,기간,자치구,세대,합계,합계.1,합계.2,한국인,한국인.1,한국인.2,등록외국인,등록외국인.1,등록외국인.2,세대당인구,65세이상고령자
0,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
1,2021.2/4,합계,"4,405,638","9,795,426","4,756,178","5,039,248","9,565,990","4,649,442","4,916,548","229,436","106,736","122,700",2.17,"1,576,971"
2,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"


CCTV 데이터 전처리

In [335]:
# 컬럼명 수정 및 행 제거
cctv.rename(columns={'구분' : '구별'}, inplace=True)
cctv.rename(columns={'총계':'CCTV대수'}, inplace=True)
cctv = cctv.drop(index=[0], axis=1)
cctv.head(3)

,구별,CCTV대수,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
3,용산구,"2,383",34,71,234,125,221,298,351,125,307,617,0


In [336]:
# 단위 기호 및 자치구명 공백 제거
for column in cctv.columns[1:]:
    cctv[column] = cctv[column].str.replace(',', '').astype(int)
cctv.구별 = cctv.구별.apply(lambda x : x.replace(' ',''))
cctv.head(3)

,구별,CCTV대수,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중구,2333,16,114,87,77,236,240,372,386,155,361,289
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0


In [337]:
# 최근증가율 컬럼 생성
cctv_2017 = cctv[['2012년 이전', '2012년','2012년','2013년','2014년','2015년','2016년','2017년']].sum(axis=1)
cctv_recent = cctv[['2018년', '2019년', '2020년']].sum(axis=1)
cctv['최근증가율(%)'] = round((cctv_recent / cctv_2017) * 100,1)
cctv.head(3)

,구별,CCTV대수,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근증가율(%)
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,20.5
2,중구,2333,16,114,87,77,236,240,372,386,155,361,289,71.8
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0,74.7


In [338]:
# 필요없는 컬럼 제거
cctv.drop(columns=['2012년 이전', '2012년','2012년','2013년','2014년','2015년','2016년','2017년','2018년', '2019년', '2020년', '2021년'], inplace=True)
cctv.head(3)

,구별,CCTV대수,최근증가율(%)
1,종로구,1772,20.5
2,중구,2333,71.8
3,용산구,2383,74.7


서울시주민등록인구 데이터 전처리

In [339]:
# 필요없는 컬럼 및 행 제거
df.drop(columns=['기간','세대','합계.1','합계.2','한국인','한국인.1','한국인.2','등록외국인.1','등록외국인.2','세대당인구'],inplace=True)
df = df.drop(index=[0,1], axis=1)
df.head(3)

,자치구,합계,등록외국인,65세이상고령자
2,종로구,"155,106","9,077","27,605"
3,중구,"132,259","9,243","23,980"
4,용산구,"240,665","14,287","38,884"


In [340]:
# 단위 기호 제거
for column in df.columns[1:]:
    df[column] = df[column].str.replace(',', '').astype(int)
df.head(3)

,자치구,합계,등록외국인,65세이상고령자
2,종로구,155106,9077,27605
3,중구,132259,9243,23980
4,용산구,240665,14287,38884


In [341]:
# 컬럼명 수정
df.rename(columns={'자치구' : '구별'}, inplace=True)
df.rename(columns={'합계' : '인구수'}, inplace=True)
df.rename(columns={'등록외국인' : '외국인'}, inplace=True)
df.rename(columns={'65세이상고령자' : '고령자'}, inplace=True)
df.head(3)

,구별,인구수,외국인,고령자
2,종로구,155106,9077,27605
3,중구,132259,9243,23980
4,용산구,240665,14287,38884


In [342]:
# 데이터프레임 합성
df2 = pd.merge(cctv, df, how='inner')
df2.set_index('구별',inplace=True)
df2.head(3)


,CCTV대수,최근증가율(%),인구수,외국인,고령자
구별,,,,,
종로구,1772,20.5,155106,9077,27605
중구,2333,71.8,132259,9243,23980
용산구,2383,74.7,240665,14287,38884


In [343]:
# 외국인비율 고령자비율 컬럼 추가
total_pop = df2[['인구수']].sum(axis=1)
pop_fore = df2[['외국인']].sum(axis=1)
pop_old = df2[['고령자']].sum(axis=1)

df2['외국인비율(%)'] = round(pop_fore / total_pop * 100,1)
df2['고령자비율(%)'] = round(pop_old / total_pop * 100,1)
df2.head(3)


,CCTV대수,최근증가율(%),인구수,외국인,고령자,외국인비율(%),고령자비율(%)
구별,,,,,,,
종로구,1772,20.5,155106,9077,27605,5.9,17.8
중구,2333,71.8,132259,9243,23980,7.0,18.1
용산구,2383,74.7,240665,14287,38884,5.9,16.2


In [344]:
df2

,CCTV대수,최근증가율(%),인구수,외국인,고령자,외국인비율(%),고령자비율(%)
구별,,,,,,,
종로구,1772,20.5,155106,9077,27605,5.9,17.8
중구,2333,71.8,132259,9243,23980,7.0,18.1
용산구,2383,74.7,240665,14287,38884,5.9,16.2
성동구,3602,43.9,295767,6605,45591,2.2,15.4
광진구,2588,107.2,355306,12825,50815,3.6,14.3
동대문구,2497,32.2,352259,14219,61262,4.0,17.4
중랑구,3296,187.1,394679,4707,70172,1.2,17.8
성북구,3958,90.9,443206,9548,73568,2.2,16.6
강북구,2462,447.0,306093,3448,63383,1.1,20.7
